In [ ]:
import requests, base64, os, json
import pandas as pd 
from io import BytesIO

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("..\Data/csvs/basal_ganglia/")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

csv_file_path = "file:///%s" % get_csv_path("experiments/cell_morphologies.csv")
df = pd.read_csv(csv_file_path)


class nmoInfo():
    def __init__(self, id, morph_id, href, img, archive):
        self.id = id
        self.morph_id = morph_id
        self.href = href
        self.img = img
        self.archive = archive
        
nmo_infos = []
        
for index, row in df.iterrows():
    nmoId = row["Neuromorpho_ID"];
    if(nmoId.startswith("NMO_00")):
        nmoId = nmoId[6:]
    elif(nmoId.startswith("NMO_0")):
        nmoId = nmoId[5:]
    else:
        nmoId = nmoId[4:]
    print(nmoId)

    
    res = requests.get("http://neuromorpho.org/api/neuron/id/" + nmoId).content
    data = json.loads(res)
    
    try:
        picture_url = data["png_url"]
        img_data = requests.get(picture_url).content
        buffered = BytesIO(img_data)
        img_base64 = base64.b64encode(buffered.getvalue())

        href = "http://neuromorpho.org/neuron_info.jsp?neuron_name=" + data["neuron_name"]
        nmo_infos.append([row["ID"], nmoId, href, img_base64, data["archive"]])
        print(len(nmo_infos), row["ID"], nmoId, href, data["archive"])
        if(nmoId == "68266"):
            print(img_base64)
    except:
        print("FAILED for", nmoId, row["Morphology_name"])
          
          
nmo_df = pd.DataFrame(nmo_infos, columns = ["id", "neuromorphoId", "href", "base64", "archive"]) 
nmo_df.to_csv("..\Data/csvs/neuromorpho/neuromorpho.csv", index=1)
          

          

8371
1 1 8371 http://neuromorpho.org/neuron_info.jsp?neuron_name=WT-P90-01 Kellendonk
830
2 2 830 http://neuromorpho.org/neuron_info.jsp?neuron_name=slm8 Jaffe
8369
3 3 8369 http://neuromorpho.org/neuron_info.jsp?neuron_name=WT-P90-03 Kellendonk
8368
4 4 8368 http://neuromorpho.org/neuron_info.jsp?neuron_name=WT-P90-04 Kellendonk
8367
5 5 8367 http://neuromorpho.org/neuron_info.jsp?neuron_name=WT-P90-05 Kellendonk
8366
